### Sql Analysis - SpaceX Dataset

In [14]:

import pandas as pd
import sqlite3
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [10]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [11]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### Select the  names of the unique launch sites in the space mission

In [16]:

# Write a SQL query to select unique launch sites
query = """
SELECT DISTINCT Launch_Site
FROM SPACEXTBL;
"""


df_unique_sites = pd.read_sql_query(query, con)

# Display the unique launch site names
print(df_unique_sites)


    Launch_Site
0   CCAFS LC-40
1   VAFB SLC-4E
2    KSC LC-39A
3  CCAFS SLC-40


### Display 5 records where launch sites begin with the string 'CCA'

In [20]:
query = """
SELECT *
FROM SPACEXTBL
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;
"""


df = pd.read_sql_query(query, con)
print(df)


         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

### Display the total payload mass carried by boosters launched by NASA (CRS)

In [22]:
query = """
SELECT SUM(Payload_Mass__kg_) AS Total_Payload_kg
FROM SPACEXTBL
WHERE Customer = 'NASA (CRS)';
"""

df = pd.read_sql_query(query, con)
print(df)


   Total_Payload_kg
0             45596


### Display average payload mass carried by booster version F9 v1.1

In [23]:
query = """
SELECT AVG(Payload_Mass__kg_) AS Avg_Payload_kg
FROM SPACEXTBL
WHERE Booster_Version = 'F9 v1.1';
"""

df = pd.read_sql_query(query, con)
print(df)

   Avg_Payload_kg
0          2928.4


### List the date when the first succesful landing outcome in ground pad was acheived.

In [24]:
query="""
Select Date 
From SPACEXTBL
WHERE Landing_Outcome = 'Success (ground pad)'
ORDER BY Date ASC
LIMIT 1;
"""

df = pd.read_sql_query(query, con)
print(df)

         Date
0  2015-12-22


### List names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [25]:
query = """
SELECT DISTINCT Booster_Version
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (drone ship)'
  AND Payload_Mass__kg_ > 4000
  AND Payload_Mass__kg_ < 6000;
"""

df = pd.read_sql_query(query, con)
print(df)

  Booster_Version
0     F9 FT B1022
1     F9 FT B1026
2  F9 FT  B1021.2
3  F9 FT  B1031.2


### List the total number of successful and failure mission outcomes

In [26]:
query = """
SELECT Mission_Outcome, COUNT(*) AS Total
FROM SPACEXTBL
GROUP BY Mission_Outcome;
"""

df = pd.read_sql_query(query, con)
print(df)

                    Mission_Outcome  Total
0               Failure (in flight)      1
1                           Success     98
2                          Success       1
3  Success (payload status unclear)      1


### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.

In [27]:
query = """
SELECT Booster_Version, Payload_Mass__kg_
FROM SPACEXTBL
WHERE Payload_Mass__kg_ = (
    SELECT MAX(Payload_Mass__kg_)
    FROM SPACEXTBL
);
"""

df = pd.read_sql_query(query, con)
print(df)

   Booster_Version  PAYLOAD_MASS__KG_
0    F9 B5 B1048.4              15600
1    F9 B5 B1049.4              15600
2    F9 B5 B1051.3              15600
3    F9 B5 B1056.4              15600
4    F9 B5 B1048.5              15600
5    F9 B5 B1051.4              15600
6    F9 B5 B1049.5              15600
7   F9 B5 B1060.2               15600
8   F9 B5 B1058.3               15600
9    F9 B5 B1051.6              15600
10   F9 B5 B1060.3              15600
11  F9 B5 B1049.7               15600


### all booster versions that carried the maximum payload mass 

In [28]:
query = """
SELECT Booster_Version, Payload_Mass__kg_
FROM SPACEXTBL
WHERE Payload_Mass__kg_ = (
    SELECT MAX(Payload_Mass__kg_)
    FROM SPACEXTBL
);
"""

df = pd.read_sql_query(query, con)
print(df)

   Booster_Version  PAYLOAD_MASS__KG_
0    F9 B5 B1048.4              15600
1    F9 B5 B1049.4              15600
2    F9 B5 B1051.3              15600
3    F9 B5 B1056.4              15600
4    F9 B5 B1048.5              15600
5    F9 B5 B1051.4              15600
6    F9 B5 B1049.5              15600
7   F9 B5 B1060.2               15600
8   F9 B5 B1058.3               15600
9    F9 B5 B1051.6              15600
10   F9 B5 B1060.3              15600
11  F9 B5 B1049.7               15600


#### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [29]:
query = """
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;
"""

df = pd.read_sql_query(query, con)
print(df)


          Landing_Outcome  Outcome_Count
0              No attempt             10
1    Success (drone ship)              5
2    Failure (drone ship)              5
3    Success (ground pad)              3
4      Controlled (ocean)              3
5    Uncontrolled (ocean)              2
6     Failure (parachute)              2
7  Precluded (drone ship)              1
